In [161]:
date.now()

Timestamp('2022-06-23 17:31:02.528603')

In [134]:
import pandas as pd
import numpy as np
import seaborn as sns
import data.collected_data.methods.get_listing_median_price as gt
import matplotlib.pyplot as plt
import re
import data.collected_data.economics.schedule as sch

sns.set(rc={'figure.figsize': (15, 10)},
        style="whitegrid", font_scale= 0.8)

path_to_values = '/Users/eaxes/DA Projects/CMC/data/collected_data/coins_date_and_values'




In [160]:
date.now()

Timestamp('2022-06-23 17:30:54.042711')

In [135]:
import pandas as pd
import numpy as np

base_df_path = "/Users/eaxes/DA Projects/CMC/data/collected_data/mined/price_token_test.csv"
values_df_path = '/Users/eaxes/DA Projects/CMC/data/collected_data/coins_date_and_values'

slug = "gamefi"
df = pd.read_csv(base_df_path, index_col=0)
token_common_df = df.query("slug == @slug")

In [136]:
token_common_df

,name,ticker,platform_raise,coingecko_id,coingecko_numerical_id,sho_price,personal_allocation,slug,median_listing_sell_price,token_personal_cap,ath_median_listing,for_charts
42,GameFi,GAFI,95000,gamefi,18292,1.0,500.0,gamefi,3.860847855497112e-17,500.0,3.860847855497112e-17,True


In [159]:
coingecko_numerical_id = int(token_common_df.coingecko_numerical_id.values[0])
coingecko_numerical_id

Timestamp('2022-06-23 17:30:33.106200')

In [138]:
token_charts = pd.read_csv(values_df_path, index_col=0).query('coingecko_numerical_id == @coingecko_numerical_id')
token_charts

,coingecko_numerical_id,date,value
12791,18292,2021-09-11 00:00,3.860848e-17
12792,18292,2021-09-12 00:00,3.860848e-17
12793,18292,2021-09-13 00:00,4.112763e-17
12794,18292,2021-09-14 00:00,3.669663e-17
12795,18292,2021-09-15 00:00,4.201025e+01
...,...,...,...
13073,18292,2022-06-20 00:00,1.042701e+01
13074,18292,2022-06-21 00:00,1.038043e+01
13075,18292,2022-06-22 00:00,1.084817e+01
13076,18292,2022-06-23 00:00,9.928408e+00


In [139]:
    def str_schedule_unit_to_arr(schedule):
        time = re.findall("[a-z]+", schedule)[0]
        quantity = re.findall("\d+", schedule)[0]
        return [quantity, time]

In [140]:
def get_tge_date(path_to_values, coingecko_id):
    print('Tge date processing...')
    try:
        id = int(coingecko_id)
        dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M')
        df_values = pd.read_csv(path_to_values, index_col=0,
                                parse_dates=['date'],
                                date_parser=dateparse)
        df_values_for_coin = df_values.query('coingecko_numerical_id == @id')
        if df_values_for_coin.empty:
            return 'no-date'
        else:
            return df_values_for_coin.date.min()
    except BaseException:
        return 'date'

In [141]:
date = get_tge_date(path_to_values, 18292)
date

Tge date processing...


Timestamp('2021-09-11 00:00:00')

In [142]:
date = date + pd.Timedelta(1, unit = 'd')


In [143]:
new_date = start_date - pd.Timedelta(dayDistance, unit="d")

NameError: name 'start_date' is not defined

In [144]:
schedule_table = sch.schedule
schedule = "Public:25% at TGE, then 25% at month 2, 4, and 6"

In [145]:
personal_cap = float(token_common_df.token_personal_cap.values[0])
personal_cap

500.0

In [146]:
table = schedule_table[schedule]
table

[['0', 0.25], ['2m', 0.25], ['2m', 0.25], ['2m', 0.25]]

In [147]:
result = []
tge_date = get_tge_date(path_to_values, 18292)
for i in range(len(table)):
    if table[i][0] == "0":
        result.append([tge_date, personal_cap * float(table[i][1])])
    else:
        if current_date > date.now() + np.timedelta64(48, 'h'):
            break
        else:
            schedule = str_schedule_unit_to_arr(table[i][0])
        previous_date = result[i - 1][0]
        current_date = previous_date + np.timedelta64(int(schedule[0]), 'M')
        result.append([current_date, personal_cap * float(table[i][1])])
        
result

Tge date processing...


[[Timestamp('2021-09-11 00:00:00'), 125.0],
 [Timestamp('2021-11-10 20:58:12'), 125.0],
 [Timestamp('2022-01-10 17:56:24'), 125.0],
 [Timestamp('2022-03-12 14:54:36'), 125.0]]

In [148]:
def get_median_price_at_date(hours, path_to_values, coingecko_id, date64):
    dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M')
    df_values = pd.read_csv(path_to_values, index_col=0,
                            parse_dates=['date'],
                            date_parser=dateparse)
    df_values = df_values.query("coingecko_numerical_id == @coingecko_id")
    border_right = date64 + np.timedelta64(hours, 'h')
    df_values = df_values.query("date < @border_right and date > @date64")
    return df_values.value.median()

In [149]:
eco = []

for stack in result:
    print(stack[0])
    median_price = get_median_price_at_date(48, path_to_values, 18292, stack[0])
    print(median_price)
    eco.append([stack[0], stack[1] * median_price])
eco

2021-09-11 00:00:00
3.860847855497112e-17
2021-11-10 20:58:12
175.122737368117
2022-01-10 17:56:24
97.66911339088853
2022-03-12 14:54:36
23.40308758372351


[[Timestamp('2021-09-11 00:00:00'), 4.826059819371389e-15],
 [Timestamp('2021-11-10 20:58:12'), 21890.342171014625],
 [Timestamp('2022-01-10 17:56:24'), 12208.639173861066],
 [Timestamp('2022-03-12 14:54:36'), 2925.385947965439]]

In [150]:
df_values = pd.read_csv(path_to_values, index_col=0).query('coingecko_numerical_id == 18292')
df_values

,coingecko_numerical_id,date,value
12791,18292,2021-09-11 00:00,3.860848e-17
12792,18292,2021-09-12 00:00,3.860848e-17
12793,18292,2021-09-13 00:00,4.112763e-17
12794,18292,2021-09-14 00:00,3.669663e-17
12795,18292,2021-09-15 00:00,4.201025e+01
...,...,...,...
13073,18292,2022-06-20 00:00,1.042701e+01
13074,18292,2022-06-21 00:00,1.038043e+01
13075,18292,2022-06-22 00:00,1.084817e+01
13076,18292,2022-06-23 00:00,9.928408e+00


In [151]:
df_values.tail(100)

,coingecko_numerical_id,date,value
12978,18292,2022-03-17 00:00,27.184496
12979,18292,2022-03-18 00:00,26.369722
12980,18292,2022-03-19 00:00,26.105279
12981,18292,2022-03-20 00:00,31.324360
12982,18292,2022-03-21 00:00,30.894085
...,...,...,...
13073,18292,2022-06-20 00:00,10.427015
13074,18292,2022-06-21 00:00,10.380431
13075,18292,2022-06-22 00:00,10.848172
13076,18292,2022-06-23 00:00,9.928408


In [152]:
type(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

str

In [153]:
date.now()

Timestamp('2022-06-23 17:29:52.395277')

In [154]:
unix_now = datetime.datetime.now().timestamp()
now = datetime.datetime.strptime('%Y-%m-%d %H:%M', str(unix_now))

ValueError: time data '%Y-%m-%d %H:%M' does not match format '1655994592.477597'

In [164]:
pd.Timestamp(datetime.datetime.now())

Timestamp('2022-06-23 17:35:11.912133')

In [162]:
type(date.now())

pandas._libs.tslibs.timestamps.Timestamp

In [165]:
result_df = pd.DataFrame(columns=['token_numerical_id', 'left_date_sale', 'right_date_sale', 'tokens_for_sale', 'median_price'])
result_df

,token_numerical_id,left_date_sale,right_date_sale,tokens_for_sale,median_price


In [169]:
def get_token_eco_df(slug, coingecko_numerical_id):
    slug = "gamefi"
    df = pd.read_csv(base_df_path, index_col=0)
    token_common_df = df.query("slug == @slug")

    coingecko_numerical_id = 18292
    token_charts = pd.read_csv(values_df_path, index_col=0).query('coingecko_numerical_id == @coingecko_numerical_id')
    schedule_table = sch.schedule
    schedule = "Public:25% at TGE, then 25% at month 2, 4, and 6"
    personal_cap = float(token_common_df.token_personal_cap.values[0])
    table = schedule_table[schedule]

    result_df = pd.DataFrame(columns=['token_numerical_id', 'sale_date',
                                      'tokens_for_sale', 'median_price'])
    result = []
    tge_date = get_tge_date(path_to_values, coingecko_numerical_id)
    for i in range(len(table)):
        if table[i][0] == "0":
            result.append([tge_date, personal_cap * float(table[i][1])])
        else:
            schedule = str_schedule_unit_to_arr(table[i][0])
            previous_date = result[i - 1][0]
            current_date = previous_date + np.timedelta64(int(schedule[0]), 'M')
            if current_date > pd.Timestamp(datetime.datetime.now()) + np.timedelta64(48, 'h'):
                break
            result.append([current_date, personal_cap * float(table[i][1])])

    eco = []

    for stack in result:
        median_price = get_median_price_at_date(48, path_to_values, 18292, stack[0])
        eco.append([stack[0], stack[1] * median_price])
        series = pd.Series([coingecko_numerical_id,
                            stack[0],
                            stack[1],
                            get_median_price_at_date(48, path_to_values, 18292, stack[0])
                            ])
        result_df = result_df.append(series, ignore_index=True)
    return result_df

get_token_eco_df("gamefi",18292)

Tge date processing...


,token_numerical_id,sale_date,tokens_for_sale,median_price,0,1,2,3
0,NaN,NaN,NaN,NaN,18292.0,2021-09-11 00:00:00,125.0,3.860848e-17
1,NaN,NaN,NaN,NaN,18292.0,2021-11-10 20:58:12,125.0,1.751227e+02
2,NaN,NaN,NaN,NaN,18292.0,2022-01-10 17:56:24,125.0,9.766911e+01
3,NaN,NaN,NaN,NaN,18292.0,2022-03-12 14:54:36,125.0,2.340309e+01
